In [1]:
import os
import pandas as pd
import numpy as np
from communities.algorithms import louvain_method
import time
import yfinance as yf
from datetime import datetime
from dateutil.relativedelta import relativedelta
from sklearn.linear_model import LinearRegression
import threading

# get correlation matrix file names
data_folder = "Data/Corr_Mat"
output_folder = "Data/Ticker_List"

## Download Data

In [2]:
def download_stock_pool_data():
    # get data directory path
    cur_path = os.path.dirname(__file__)
    data_directory_path = os.path.relpath('../Data', cur_path)

    # get latest russell 1000 constituents
    russell1000_info = pd.read_excel(
        io=data_directory_path + "\\Russell_1000_Constituents_20221007.xlsx",
        sheet_name="Holdings",
        skiprows=range(7))
    # get list of tickers
    stocks_pool_list = list(russell1000_info.Ticker.values)
    # get historical market data of current Russel 1000 constituents
    stocks_pool_data = yf.Tickers(stocks_pool_list).history(start="2012-01-01")["Close"]

    # get historical market data of S&P500
    sp500_data = pd.DataFrame(yf.Ticker("^GSPC").history(start="2012-01-01")["Close"])
    # rename S&P 500 data column
    sp500_data.columns = ["SP500"]

    # merge two dataframes
    raw_data = stocks_pool_data.join(sp500_data)

    # drop stocks with more than 1000 NaNs
    raw_data = raw_data.dropna(axis="columns", thresh=2500)

    # save stock pool data into csv
    raw_data.to_csv(data_directory_path + "\\Raw_Data_20221007.csv")

# # get stock pool data from yahoo finance
# download_stock_pool_data()

## Generate Correlation Matrices

In [3]:
def generate_residual_matrices():
    # get data directory path
    cur_path = os.path.dirname(__file__)
    data_directory_path = os.path.relpath('../Data', cur_path)
    # read data from file
    raw_data = pd.read_csv(data_directory_path + "\\Raw_Data_20221007.csv", index_col=0)

    # set rebalancing frequency: every month
    rebalance_freq_period = relativedelta(months=1)
    # set business day convention for rebalancing
    business_day_convention = "Modified Following"
    # training set length
    train_set_length_period = relativedelta(months=6)
    # set date range
    first_date = datetime.strptime(raw_data.index[0], "%Y-%m-%d")
    last_date = datetime.strptime(raw_data.index[-1], "%Y-%m-%d")

    # initialize date range
    train_start_date = first_date
    train_end_date = train_start_date + train_set_length_period - relativedelta(days=1)

    test_start_date = train_end_date + relativedelta(days=1)
    test_end_date = test_start_date + rebalance_freq_period

    # traverse the data set
    while test_end_date < last_date:
        # do regression
        temp_train_data = raw_data.loc[train_start_date.__str__()[:10]:test_start_date.__str__()[:10], :]
        # save the residuals
        temp_residuals = pd.DataFrame(index=temp_train_data.index, columns=temp_train_data.columns.drop("SP500"))
        for ticker in temp_residuals.columns:
            # get data
            y_x = temp_train_data[[ticker, "SP500"]]
            # drop nas
            y_x = y_x.dropna(axis="index", how="any")
            # rename columns
            y_x.columns = ["y", "x"]
            # whether there's sufficient trading days
            if len(y_x.index) < len(temp_train_data) * 0.9:
                continue
            else:
                # calculate returns
                y_x = np.log(y_x).diff().dropna(axis="index", how="any")
                y = np.array(y_x["y"])
                x = np.array(y_x["x"]).reshape(-1, 1)
                # do regression
                reg = LinearRegression(fit_intercept=True).fit(x, y)
                # calculate residual
                y_x.loc[:, "res"] = np.subtract(y, (reg.intercept_ - reg.coef_[0] * x)[:, 0])
                # add residual to temp_residuals
                temp_residuals.loc[y_x.index, ticker] = y_x.res
        # drop nans in dataframe
        temp_residuals = temp_residuals.dropna(axis="index", how="all")
        temp_residuals = temp_residuals.dropna(axis="columns", how="any")
        # calculate correlations
        temp_residuals = temp_residuals.astype(float)
        temp_corr_matrix = temp_residuals.corr()

        # save the matrix to file
        temp_corr_matrix.to_csv(data_directory_path + "\\Corr_Mat\\" + train_start_date.__str__()[:10] + ".csv")

        # update dates
        train_start_date += rebalance_freq_period
        test_start_date += rebalance_freq_period
        test_end_date += rebalance_freq_period

# # calculate correlation between residuals of stocks during 6M period
# generate_residual_matrices()

## Split Graph Into Communities

In [4]:
def get_communities(file_name:str, input_folder:str = data_folder):
    print(file_name)
    # get correlation matrix
    corr_mat = pd.read_csv(input_folder+"/"+file_name, index_col=0)
    # convert to adjacency matrix
    adj_mat = np.abs(corr_mat - np.diag(np.diag(corr_mat)))
    # using Louvain method to split the graph into 20 communities
    communities, _ = louvain_method(adj_mat.values, 20)
    # get ticker list of each community
    cluster_list = []
    for community in communities:
        cluster = list(community)
        cluster_list.append(adj_mat.columns[cluster])
    # save the list to txt
    global output_folder
    with open(output_folder + "/"+file_name.replace(".csv",".txt"), 'w') as f:
        for cluster in cluster_list:
            f.write(str(list(cluster)) + "\n")

In [5]:
class MyThread(threading.Thread):

    def __init__(self, func, arg):
        super().__init__()
        self.func = func
        self.arg = arg

    def run(self):
        self.func(*self.arg)

In [6]:
def get_all_communities(file_list:list[str] = os.listdir(data_folder), input_folder:str = data_folder, thread_num:int = 8):
    # create 8 threads
    for i in range(0, len(file_list), thread_num):
        thread_list = []
        print("Iteration {} start, current time = {}".format(i//8+1, datetime.now()))
        for j in range(thread_num):
            file_name = file_list[i + j]
            thread = MyThread(get_communities, (file_name, input_folder))
            thread.start()
            thread_list.append(thread)
        for thread in thread_list:
            thread.join()

# generate clusters of tickers and save them to txt file
# file_name_list = os.listdir(data_folder)
# get_all_communities(file_name_list, data_folder, 8)

## Get Centroids of Clusters

In [13]:
def unwrap_list_from_str(str_list:str)->list[str]:
    return str_list.replace("\'","").replace("[","").replace("]","").replace("\n","").replace(" ", "").split(",")

def get_tickers_of_clusters():
    # read clusters from file
    file_name_list =os.listdir(output_folder)
    df_cluster_data = pd.DataFrame(columns=["cluster_{}".format(i) for i in range(20)])
    for file_name in file_name_list:
        # read string from file
        with open(output_folder+"/"+file_name, "r") as f:
            list_str_data = f.readlines()
        list_data = []
        for str_data in list_str_data:
            list_data.append(unwrap_list_from_str(str_data))
        df_cluster_data.loc[file_name[:10],:] = list_data
    return df_cluster_data

# # save clusters as a dataframe
# df_cluster_data = get_tickers_of_clusters()
# # save to local
# df_cluster_data.to_csv("Data/Stock_Clusters.csv")

C:\Users\77026\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\internals\blocks.py:937: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr_value = np.asarray(value)


In [20]:
def select_stock_from_clusters():
    # read data from local
    df_cluster_data = pd.read_csv("Data/Stock_Clusters.csv",index_col=0)
    # convert string to list
    df_cluster_data = df_cluster_data.applymap(lambda x: unwrap_list_from_str(x))
    # convert index to train_end_data
    train_set_length_period = relativedelta(months=6)
    df_cluster_data.index = df_cluster_data.index.map(lambda x: datetime.strptime(x, "%Y-%m-%d")+train_set_length_period)

    # read stock price data from locals
    performance_data = pd.read_csv(io="Data/Raw_Data_20221007.csv", index_col=0)


    return df_cluster_data

select_stock_from_clusters()

,cluster_0,cluster_1,cluster_2,cluster_3,cluster_4,cluster_5,cluster_6,cluster_7,cluster_8,cluster_9,cluster_10,cluster_11,cluster_12,cluster_13,cluster_14,cluster_15,cluster_16,cluster_17,cluster_18,cluster_19
2012-07-03,"[A, PEGA, ABMD, ACN, ADBE, ADI, PKG, PKI, ADSK...","[AA, PDCE, PFG, ACM, PH, ADM, PLUG, AGCO, AGO,...","[AAL, FDX, EXP, DAL, UAL, LUV, ASH, CPA, DHI, ...","[AAP, AAPL, PENN, PHM, PII, AGNC, POOL, AIG, P...","[PCG, PEAK, PEG, ABC, PEP, ABT, PFE, PG, ADP, ...","[ACGL, ACHC, PGR, PNC, AFG, PNR, ALK, ALL, AMG...","[BALL, RGEN, GGG, LSTR, DD, HRB, CCK, MORN, SL...","[CI, CRL, UNH, CTAS, AMT, APO, VRTX, HUM, BA, ...",[AMZN],[AN],[ANSS],[AOS],[APD],[AXP],"[BAX, BIO]","[EEFT, BIIB, DGX, BMRN, CHDN]","[BOKF, BK]","[BLDR, CACC, CSGP, EXEL]",[CNC],[DECK]
2012-08-03,"[A, AA, PCAR, AAP, AAPL, PEGA, ACN, ADBE, ADI,...","[AAL, ACGL, ACHC, PFG, PGR, AFG, AFL, PNC, PNF...","[PCG, PEG, ABC, ABT, PEP, PFE, PG, ADM, ADP, A...","[PEAK, ABMD, PENN, PHM, PKG, PLD, PLUG, AGNC, ...","[PDCE, TMUS, CLR, ACM, CMCSA, TRGP, LUMN, COHR...","[CINF, CLX, CPA, AJG, HAS, RE, AON, MMC, AXS, ...",[AN],[AOS],[AVY],[AWI],[AZO],"[BALL, BIO]","[BG, CL]","[INCY, BMY, MRCY]",[BRKR],[CASY],[CHDN],[CNC],[CSGP],[DGX]
2012-09-03,"[A, AA, PCAR, PEGA, ACM, ACN, ADBE, ADI, PH, A...","[PB, AAL, ACGL, ACHC, PFG, PGR, AFG, AFL, PNC,...","[AAP, AAPL, PENN, PHM, PII, POOL, PPG, ALB, PV...","[PCG, PEAK, ABC, ABMD, ABT, PEG, PEP, PFE, PG,...","[ALL, AXS, BWXT, BYD, AON, CB, APD, CBRE, CCL,...","[AMP, BG, AKAM]",[ALGN],[ALNY],"[DLB, AMAT, AME]","[FISV, FLT, AMGN, UTHR, MRCY, DISH, MTG, BAH, ...","[AMZN, BKNG]",[ANSS],"[CPRT, AWI, AOS]","[EQT, PDCE, CLR, CTRA, DVN, APA, DE, BKR, EOG]","[CE, AVY]","[CACC, AYI]",[AZTA],"[BLDR, CLF, CHRW, BBY]",[CCK],[CNC]
2012-10-03,"[A, AA, PCAR, AAPL, PEGA, ACM, ACN, ADBE, ADI,...","[AAL, PB, PEAK, ACGL, ACHC, PENN, PFG, PHM, AF...","[PARA, AAP, TJX, TNL, LOW, TRIP, PII, LSTR, TS...","[PAYX, PCG, ABC, ABMD, ABT, PEG, PEP, PFE, PG,...","[PDCE, CLF, CLR, CME, GE, COO, COP, AIZ, HAL, ...",[AVGO],[AVY],[AWI],"[AXON, CASY]",[AXP],"[FICO, BSX, AXS, HCA, CB, EFX, GILD, IPG, MMC,...","[SLGN, CHRW, AZPN]","[FISV, LH, CL, CNC, CPB, MA, MDT, QGEN, UTHR, ...","[CDNS, BG, DFS, MCO]","[CLH, FBHS, EXPE, BKNG]",[BR],[BX],[CACC],"[CHH, CAR]","[ELV, CI, UNH, HUM]"
2012-11-03,"[A, AA, PCAR, PDCE, ACM, ACN, ADBE, ADI, PFG, ...","[AAL, AAP, AAPL, PEAK, ACHC, PENN, PHM, PII, P...","[ABC, ABMD, ABT, ACGL, PCG, PEG, PEP, PFE, ADM...","[PAYX, CIEN, LKQ, PEGA, FTNT, PNR, CPRI, ALGN,...","[CHE, FISV, LH, TNL, CNC, POST, ALNY, MASI, MC...","[AWI, BLDR, CPRT]","[CHH, AXON, NEU, CTRA, CMCSA, EXPE, BKNG, COLM...",[AXP],"[BR, FICO, AXS, CINF, STE, CB]","[PAG, PARA, CME, CMG, TPR, LULU, GME, TTEK, M,...","[NYT, BBY]","[TFC, PB, FNB, SBNY, CBSH, CFR, BOH, BOKF]","[BYD, CRUS]",[CACC],[CASY],[CHDN],[CHRW],"[GOOG, GOOGL]",[KMI],[RGLD]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-03,"[A, ORCL, ORLY, ABT, PANW, ACN, ADBE, ADI, PAY...",[AA],"[AAL, PH, PII, AL, ALK, PPC, PPG, PVH, APTV, A...","[AOS, AAP, AZO]",[AAPL],[ABC],[ABMD],"[ORI, OSK, OVV, OXY, OZK, PACW, ACGL, PAG, PAR...",[ACHC],[ACM],[ADM],"[PCG, PEAK, AEE, AEP, PEG, PEP, PFE, PG, PGR, ...",[AES],[AGCO],[AGNC],[AN],[APA],[ATVI],[ZG],[ZION]
2022-06-03,[A],[AA],"[ALK, AAL]","[AAP, CRI, GPC, DLTR, AZO, BBY, AOS]",[AAPL],[ABC],"[ORCL, ABMD, PANW, PARA, ADBE, ADI, ADSK, PEGA...","[ACN, ADP, ABT]","[ORI, OSK, OZK, PACW, ACGL, PAG, PB, PCAR, PCG...",[ACHC],[ACM],[ADM],"[ORLY, PAYX, PEAK, AEE, AEP, PEG, PEP, PFE, PG...",[AGCO],[AGNC],[AN],"[SWN, OVV, OXY, CLR, FMC, PDCE, LNG, TPL, COP,...",[ATVI],[ZG],[ZION]
2022-07-03,"[A, ORCL, AAL, AAPL, ABMD, PANW, ADBE, ADI, AD...","[AA, ORI, OSK, OVV, OXY, OZK, PACW, PAG, PARA,...","[ORLY, AAP, PAYX, PEAK, AEE, AEP, AES, PEG, PE...",[ABC],[ABT],[ACGL],[ACHC],[ACM],[ACN],[ADP],[AGNC],[AJG],"[ALK, AMG, BA, BEN, BERY, APTV, CCL]",[AMCR],[AME],[AN],"[GPC, DLTR, AZO, BBY, AOS, CASY, NWL, DCI]",[APA],[APD],[ATVI]
2022-08-03,"[A, ABT]",[AA],"

In [22]:
performance_data = pd.read_csv("Data/Raw_Data_20221007.csv", index_col=0)
performance_data

,A,AA,AAL,AAP,AAPL,ABC,ABMD,ABT,ACGL,ACHC,...,XPO,XRAY,XYL,Y,YUM,ZBH,ZBRA,ZG,ZION,SP500
Date,,,,,,,,,,,,,,,,,,,,,
2012-01-03,23.816809,20.847895,4.826837,66.313416,12.629211,32.344158,18.250000,22.094467,12.493333,9.920000,...,6.891735,33.225037,21.854513,274.094543,34.621609,47.885345,35.720001,7.193639,14.626753,1277.060059
2012-01-04,23.627478,21.344809,4.741990,67.868080,12.697083,32.454601,18.110001,22.008778,12.266667,9.740000,...,6.821886,33.215645,22.576048,274.229248,34.858082,47.237286,35.450001,7.174930,14.949463,1277.300049
2012-01-05,24.156307,21.141527,5.156795,68.299934,12.838050,32.700989,18.270000,21.958130,12.500000,9.770000,...,6.827707,33.243820,22.254400,275.114532,35.124077,47.796570,35.400002,7.171812,15.368120,1281.060059
2012-01-06,24.417461,20.689783,5.279352,68.616646,12.972254,32.785950,18.139999,21.759474,12.486667,11.360000,...,6.885914,33.046597,21.880594,275.624481,35.378242,47.920841,35.110001,7.184284,15.385561,1277.810059
2012-01-09,25.057272,21.299643,5.392481,68.597466,12.951676,32.768955,18.209999,21.755571,12.396667,11.300000,...,6.810244,32.886967,21.967527,274.181152,35.242294,48.275951,34.950001,7.181166,15.594892,1280.699951
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-03,126.379997,37.389999,11.920000,163.369995,142.449997,139.309998,255.149994,99.519997,46.330002,81.050003,...,46.740002,29.180000,90.199997,843.289978,109.110001,107.320000,272.089996,29.510000,52.209999,3678.429932
2022-10-04,131.410004,40.230000,12.950000,167.830002,146.100006,142.729996,264.010010,103.080002,48.689999,83.879997,...,49.080002,29.930000,93.139999,843.250000,111.410004,111.690002,281.649994,31.549999,54.520000,3790.929932
2022-10-05,132.639999,40.080002,12.870000,167.639999,146.399994,142.449997,264.260010,103.379997,47.959999,83.120003,...,49.279999,30.240000,92.660004,843.109985,112.330002,112.489998,281.880005,30.129999,53.520000,3783.280029
